In [130]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

## Load the data into a DataFrame
djs = pd.read_csv('assets/Iowa_Liquor_sales_sample_10pct.csv', encoding='latin-1')
df = pd.read_csv('assets/iowa_15.csv', encoding='latin-1')

In [131]:
djs.dtypes

Date                      object
Store Number               int64
City                      object
Zip Code                  object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Item Number                int64
Item Description          object
Bottle Volume (ml)         int64
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold               int64
Sale (Dollars)            object
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

In [132]:
#Converts State Bottle Cost Series to Float
djs['State Bottle Cost'] = (djs['State Bottle Cost'].replace( '[\$,)]','', regex=True )\
                            .replace( '[(]','-',   regex=True ).astype(float))

#Converts State Bottle Retail Series to Float
djs['State Bottle Retail'] = (djs['State Bottle Retail'].replace( '[\$,)]','', regex=True )\
                              .replace( '[(]','-',   regex=True ).astype(float))

#Converts Sale Dollars Series to Float
djs['Sale (Dollars)'] = (djs['Sale (Dollars)'].replace( '[\$,)]','', regex=True )\
                         .replace( '[(]','-',   regex=True ).astype(float))

In [214]:
djs.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.0,5.55


In [133]:
type(djs['Sale (Dollars)'][5])

numpy.float64

In [134]:
djs.columns

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')

In [185]:
print djs['City'].nunique()
print df['NAME'].nunique()

hanks = pd.pivot_table(df, index=['NAME'], values=['POPESTIMATE2015'])
hanks.head()

385
1210


,POPESTIMATE2015
NAME,
Ackley city,1560.0
Ackley city (pt.),780.0
Ackworth city,86.0
Adair County,7228.0
Adair city,728.0


In [330]:
name_df = df.iloc[:,8:9]
pop_df = df.iloc[:,17:]

testee = pd.concat([name_df, pop_df], axis=1)
testee

testee['NAME'] = (testee['NAME'].replace( ' city','', regex=True ))
testee




#The following code is junk.  Do not use until after Halloween
i=True
j=0
butts = []
mutts = []
for x in range(1,2161):
    for y in testee:
        if j==0:
            if "(pt.)" in testee[y][x]:
                print testee[y][x]
                j=1
        if i == True:
            butts.append(testee[y][x])
            i = False
            j=1
        else:
            mutts.append(testee[y][x])
            i=True
            j=1
    j=0
mut = pd.Series(mutts).rename("POPULATION")
but = pd.Series(butts).rename("NAME")


dtestf = pd.concat([but, mut], axis=1)
dtestf

Adair (pt.)
Casey (pt.)
Stuart (pt.)
Lenox (pt.)
Postville (pt.)
Moravia (pt.)
Walford (pt.)
Janesville (pt.)
Jesup (pt.)
Sheldahl (pt.)
Woodward (pt.)
Janesville (pt.)
Sumner (pt.)
Fairbank (pt.)
Jesup (pt.)
Stanley (pt.)
Greene (pt.)
Farnhamville (pt.)
Lytton (pt.)
Coon Rapids (pt.)
Ralston (pt.)
Durant (pt.)
West Branch (pt.)
Wilton (pt.)
Nora Springs (pt.)
Nashua (pt.)
Protivin (pt.)
Edgewood (pt.)
Postville (pt.)
Maquoketa (pt.)
Dunlap (pt.)
Clive (pt.)
Granger (pt.)
Grimes (pt.)
Urbandale (pt.)
West Des Moines (pt.)
Woodward (pt.)
Dyersville (pt.)
Edgewood (pt.)
Cascade (pt.)
Dyersville (pt.)
Zwingle (pt.)
Fairbank (pt.)
Stanley (pt.)
Sumner (pt.)
Greene (pt.)
Nashua (pt.)
Nora Springs (pt.)
Ackley (pt.)
Dows (pt.)
Shenandoah (pt.)
Tabor (pt.)
Ralston (pt.)
Adair (pt.)
Casey (pt.)
Coon Rapids (pt.)
Stuart (pt.)
Stratford (pt.)
Forest City (pt.)
Ackley (pt.)
Dunlap (pt.)
Coppock (pt.)
Protivin (pt.)
Riceville (pt.)
Gilmore City (pt.)
Lu Verne (pt.)
North English (pt.)
Victor (pt.)

,NAME,POPULATION
0,Ackley,1560
1,Ackworth,86
2,Adair,728
3,Adel,4245
4,Afton,829
5,Agency,641
6,Ainsworth,571
7,Akron,1450
8,Albert City,688
9,Albia,3829


In [136]:
i=0
for x in djs['County Number']:
    if x > 0:
        i+=1
print "Nulls:", 270954 - i

Nulls: 1076


In [139]:
t_hanks = pd.pivot_table(djs, index=['County'], values=['County Number'])
t_hanks.head()

,County Number
County,
Adair,1.0
Adams,2.0
Allamakee,3.0
Appanoose,4.0
Audubon,5.0


In [140]:
djs['County'].unique()

array([u'Bremer', u'Scott', u'Black Hawk', u'Story', u'Wright', u'Linn',
       u'Wapello', u'Cerro Gordo', u'Polk', u'Benton', u'Mahaska', u'Lee',
       u'Johnson', u'Jasper', u'Tama', u'Sioux', u'Dickinson',
       u'Pottawattamie', u'Warren', u'Webster', u'Boone', u'Jackson',
       u'Carroll', u'Hamilton', u'Woodbury', u'Washington', u'Floyd',
       u'Delaware', u"O'Brien", u'Buena Vista', u'Grundy', u'Dubuque',
       u'Adair', u'Crawford', u'Henry', u'Guthrie', u'Winneshiek', u'Clay',
       u'Muscatine', u'Taylor', u'Harrison', u'Hardin', u'Kossuth',
       u'Dallas', u'Decatur', u'Clinton', u'Butler', u'Des Moines',
       u'Calhoun', u'Fayette', u'Jones', nan, u'Hancock', u'Iowa',
       u'Howard', u'Appanoose', u'Jefferson', u'Emmet', u'Marshall',
       u'Page', u'Cedar', u'Clayton', u'Sac', u'Monona', u'Allamakee',
       u'Winnebago', u'Poweshiek', u'Marion', u'Buchanan', u'Franklin',
       u'Cherokee', u'Humboldt', u'Greene', u'Keokuk', u'Plymouth',
       u'Shelby', u

In [141]:
djs['Sale (Dollars)'].head()

0     81.00
1     41.26
2    453.36
3     85.50
4    129.60
Name: Sale (Dollars), dtype: float64

In [142]:
df.columns

Index([u'SUMLEV', u'STATE', u'COUNTY', u'PLACE', u'COUSUB', u'CONCIT',
       u'PRIMGEO_FLAG', u'FUNCSTAT', u'NAME', u'STNAME', u'CENSUS2010POP',
       u'ESTIMATESBASE2010', u'POPESTIMATE2010', u'POPESTIMATE2011',
       u'POPESTIMATE2012', u'POPESTIMATE2013', u'POPESTIMATE2014',
       u'POPESTIMATE2015'],
      dtype='object')